In [54]:
from langchain_community.utilities import WikipediaAPIWrapper,DuckDuckGoSearchAPIWrapper,SearxSearchWrapper
from langchain_community.tools import WikipediaQueryRun,DuckDuckGoSearchRun,SearxSearchRun

In [55]:
wiki_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
duck_wrapper=DuckDuckGoSearchAPIWrapper(max_results=1,source='news')
search_wrap=SearxSearchWrapper(searx_host="http://127.0.0.1:8888")

In [56]:
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper)
duck=DuckDuckGoSearchRun(api_wrapper=duck_wrapper)
search = SearxSearchRun(wrapper=search_wrap, response_format='content_and_artifact',description="Use this tool to perform a general web search when other tools don't have sufficient information.")

In [57]:
tools=[wiki,duck,search]

In [58]:
from langchain_ollama import ChatOllama
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool

In [59]:
model = ChatOllama(model="mistral")

In [60]:
memory = InMemoryChatMessageHistory(session_id="test-session")

In [61]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        # First put the history
        ("placeholder", "{chat_history}"),
        # Then the new input
        ("human", "{input}"),
        # Finally the scratchpad
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [62]:
tools = tools

In [63]:
agent = create_tool_calling_agent(model, tools, prompt)
initialize_agent(
    tools,
    model,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # best for chat-based + memory
    verbose=True
)

AgentExecutor(verbose=True, tags=['conversational-react-description'], agent=ConversationalAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={}, partial_variables={}, template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additio

In [64]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [65]:
config = {"configurable": {"session_id": "test-session"}}

In [ ]:
agent_with_chat_history.invoke(
        {"input": "Search the web for latest trends in AI."}, config
    )['output']

In [ ]:
agent_with_chat_history.invoke(
        {"input": "What did I just ask before this?"}, config
    )['output']

' Before this, you asked for the latest trends in Artificial Intelligence (AI).'